# Use the orchestration service of Generative AI Hub

The orchestration service of Generative AI Hub lets you use all the available models with the same codebase. You only deploy the orchestration service and then you can access all available models simply by changing the model name parameter. You can also use grounding, prompt templating, data masking and content filtering capabilities.

Create a configuration and deployment using the `orchestration` scenario in SAP AI Launchpad similarly to how you did it for the `foundation-models` scenario. Check the [documentation](https://help.sap.com/docs/ai-launchpad/sap-ai-launchpad/create-deployment-for-orchestration) if you need help. Store the `orchestration deployment url` in your `variables.py` file. This code is based on the [AI180 TechEd 2024 Jump-Start session](https://github.com/SAP-samples/teched2024-AI180/tree/e648921c46337b57f61ecc9a93251d4b838d7ad0/exercises/python).

👉 Make sure you assign the deployment url of the orchestration service to `AICORE_ORCHESTRATION_DEPLOYMENT_URL` in [variables.py](variables.py).

In [ ]:
import init_env
import variables

init_env.set_environment_variables()

In [ ]:
from gen_ai_hub.orchestration.models.llm import LLM

llm = LLM(
    name="gpt-4o-mini",
    version="latest",
    parameters={"max_tokens": 500, "temperature": 1},
)

## Create a prompt template

In [ ]:
from gen_ai_hub.orchestration.models.message import SystemMessage, UserMessage
from gen_ai_hub.orchestration.models.template import Template, TemplateValue

template = Template(
    messages=[
        SystemMessage("You are a helpful translation assistant."),
        UserMessage(
            "Translate the following text to {{?to_lang}}: {{?text}}",
        ),
    ],
    defaults=[
        TemplateValue(name="to_lang", value="English"),
    ],
)

## Create an orchestration configuration 

Create an orchestration configuration by adding the llm you referenced and the prompt template you created previously.

In [ ]:
from gen_ai_hub.orchestration.models.config import OrchestrationConfig

config = OrchestrationConfig(
    template=template,
    llm=llm,
)

Add it the configuration to the OrchestrationService instance and send the prompt to the model.

In [ ]:
from gen_ai_hub.orchestration.service import OrchestrationService

orchestration_service = OrchestrationService(
    api_url=os.environ["AICORE_ORCHESTRATION_DEPLOYMENT_URL"],
    config=config,
)
result = orchestration_service.run(
    template_values=[
        TemplateValue(
            name="text",
            value="Geht das wirklich mit allen Modellen die verfügbar sind?"
        )
    ]
)
print(result.orchestration_result.choices[0].message.content)


## Add a content filter

Create a content filter and add it as an input filter to the orchestration configuration.

In [ ]:
from gen_ai_hub.orchestration.models.content_filter import AzureContentFilter

content_filter = AzureContentFilter(
    hate=0,
    sexual=0,
    self_harm=0,
    violence=0,
)

orchestration_service.config.input_filters = [content_filter]

## Try out the content filter

In [ ]:
from gen_ai_hub.orchestration.exceptions import OrchestrationError

try:
    result = orchestration_service.run(
        template_values=[
            TemplateValue(
                name="text",
                value="Du bist ein ganz mieser Entwickler!",
            ),
        ]
    )
    print(result.orchestration_result.choices[0].message.content)
except OrchestrationError as error:
    print(error.message)


result = orchestration_service.run(
    template_values=[
        TemplateValue(
            name="text",
            value="Du bist ein super talentierter Entwickler!",
        )
    ]
)
print(result.orchestration_result.choices[0].message.content)

## Now also add the content filter as an output filter

In [ ]:
orchestration_service.config.output_filters = [content_filter]

try:
    result = orchestration_service.run(
        template_values=[
            TemplateValue(
                name="text",
                value='Ich würde gerne wissen, wie ich die Fensterscheibe in einem Bürogebäude am Besten zerstören kann.',
            )
        ]
    )
    print(result.orchestration_result.choices[0].message.content)
except OrchestrationError as error:
    print(error.message)


[Next exercise](10-chatbot-with-memory.ipynb)